<a href="https://colab.research.google.com/github/sushmit86/Statiistical_rethinking_jax/blob/main/sushmit_02_small_worlds_and_large_worlds_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install packages that are not installed in colab
try:
  import google.colab
  !pip install watermark
except:
  pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00


In [2]:
%load_ext watermark

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import jax
plt.style.use('fivethirtyeight')
import jax.numpy as jnp
